# ASL Project

In [3]:
#Import libraries
import numpy as np
import random

# Imports to view data
import cv2
import pandas as pd
from glob import glob

# Visualization
from keras.utils import print_summary
from matplotlib import pyplot as plt

#ML libraries
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint  
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

In [4]:
#Directory paths
TRAIN_DIR = "../Dataset/asl_alphabet_train/asl_alphabet_train"
TEST_DIR = "../Dataset/asl_alphabet_test"
MODEL_DIR = './Model'
MODEL_PATH = MODEL_DIR+"/Model1-xception.h5"
MODEL_WEIGHT_PATH = MODEL_DIR+"/Model_Weight1-xception.h5"

In [5]:
# Set global variables
TARGET_SIZE = (64, 64)
TARGET_DIMS = (64, 64, 3) # add channel for RGB
CLASSES = 29
VALIDATION_SPLIT = 0.1
BATCH_SIZE = 64
LABELS_NAME = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q',
               'R','S','T','U','V','W','X','Y','Z','del','nothing','space']

## Data Augmentation

In [6]:
#Load Train dataset
train_image_generator = ImageDataGenerator(
    samplewise_center=True,
    samplewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=VALIDATION_SPLIT
)

validation_image_generator = ImageDataGenerator(
    samplewise_center=True,
    samplewise_std_normalization=True,
    validation_split=VALIDATION_SPLIT
)

train_generator = train_image_generator.flow_from_directory(TRAIN_DIR, target_size=TARGET_SIZE, batch_size=BATCH_SIZE, shuffle=True, subset="training")
val_generator = validation_image_generator.flow_from_directory(TRAIN_DIR, target_size=TARGET_SIZE, batch_size=BATCH_SIZE, subset="validation")

Found 78300 images belonging to 29 classes.
Found 8700 images belonging to 29 classes.


## Model

In [6]:
#Define Model Xception Model
def Xception_model_build():
    from keras.applications.xception import Xception
    from keras.layers import Input

    input_tensor = Input(shape=TARGET_DIMS)
    model = Xception(input_tensor = input_tensor, weights=None, include_top=True, classes= CLASSES)
    return model

In [7]:
from keras.models import Model
base_model = Xception_model_build()
x = base_model.output
#x = (Dense(CLASSES, activation='softmax'))(x)
model = Model(inputs=base_model.input, outputs=x)

#Compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
model.summary()
model.save(MODEL_PATH)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 31, 31, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 31, 31, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 31, 31, 32)   0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [10]:
#Checkpointer to save the best models
checkpointer = ModelCheckpoint(filepath=MODEL_WEIGHT_PATH, 
                               verbose=1, save_best_only=True)

steps_per_epoch = int( np.ceil(len(train_generator)*2 / BATCH_SIZE) )
validation_steps = int( np.ceil(len(val_generator)*2 / BATCH_SIZE) )

model.fit_generator(train_generator, validation_data=val_generator, 
                    steps_per_epoch =  steps_per_epoch,
                    validation_steps = validation_steps,
                    epochs=10, callbacks=[checkpointer], verbose=1)

Epoch 1/10
39/39 [==============================] - 264s 7s/step - loss: 3.6387 - acc: 0.0565 - val_loss: 3.5550 - val_acc: 0.0219

Epoch 00001: val_loss improved from inf to 3.55501, saving model to ./Model/Model_Weight1-xception.h5
Epoch 2/10
39/39 [==============================] - 270s 7s/step - loss: 3.3899 - acc: 0.0505 - val_loss: 3.6264 - val_acc: 0.0625

Epoch 00002: val_loss did not improve from 3.55501
Epoch 3/10
39/39 [==============================] - 271s 7s/step - loss: 3.2409 - acc: 0.0733 - val_loss: 15.6648 - val_acc: 0.0281

Epoch 00003: val_loss did not improve from 3.55501
Epoch 4/10
39/39 [==============================] - 274s 7s/step - loss: 2.9895 - acc: 0.1162 - val_loss: 15.3122 - val_acc: 0.0500

Epoch 00004: val_loss did not improve from 3.55501
Epoch 5/10
39/39 [==============================] - 274s 7s/step - loss: 2.7829 - acc: 0.1575 - val_loss: 15.3626 - val_acc: 0.0469

Epoch 00005: val_loss did not improve from 3.55501
Epoch 6/10
39/39 [=============

In [11]:
#Predict on validation dataset
predictions = model.predict_generator(val_generator, steps=1)        
predictions = np.argmax(predictions, axis=-1) #multiple categories
label_map = (train_generator.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v

predictions = [label_map[k] for k in predictions]

print(predictions, len(predictions))

loss, acc = model.evaluate_generator(val_generator, steps=1, verbose=0)

print(loss,",",acc)

['W', 'nothing', 'W', 'W', 'space', 'del', 'W', 'del', 'del', 'F', 'del', 'del', 'space', 'space', 'W', 'W', 'K', 'W', 'E', 'F', 'W', 'W', 'S', 'space', 'del', 'F', 'W', 'del', 'W', 'W', 'W', 'F', 'nothing', 'E', 'nothing', 'W', 'W', 'nothing', 'W', 'E', 'K', 'space', 'space', 'F', 'K', 'del', 'nothing', 'F', 'G', 'H', 'space', 'del', 'space', 'W', 'space', 'space', 'nothing', 'F', 'E', 'E', 'F', 'W', 'del', 'E'] 64
8.709136962890625 , 0.171875


In [12]:
test_image_generator = ImageDataGenerator(
    samplewise_center = True,
    samplewise_std_normalization = True,
)

test_generator = test_image_generator.flow_from_directory(TEST_DIR, target_size=TARGET_SIZE, batch_size=28, shuffle=False, 
    class_mode='categorical')
print(test_generator.classes)

Found 28 images belonging to 1 classes.
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [13]:
#Predict
test_generator.reset()
predictions = model.predict_generator(test_generator, steps=1)
predictions = np.argmax(predictions, axis=1) #multiple categories
label_map = (train_generator.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v

predictions = [label_map[k] for k in predictions]

print(predictions, len(predictions))

['del', 'F', 'F', 'F', 'E', 'F', 'G', 'del', 'R', 'H', 'F', 'K', 'G', 'N', 'B', 'nothing', 'W', 'space', 'space', 'space', 'space', 'W', 'F', 'W', 'space', 'space', 'nothing', 'W'] 28


In [14]:
filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
print(results)

                              Filename Predictions
0         asl_alphabet_test\A_test.jpg         del
1         asl_alphabet_test\B_test.jpg           F
2         asl_alphabet_test\C_test.jpg           F
3         asl_alphabet_test\D_test.jpg           F
4         asl_alphabet_test\E_test.jpg           E
5         asl_alphabet_test\F_test.jpg           F
6         asl_alphabet_test\G_test.jpg           G
7         asl_alphabet_test\H_test.jpg         del
8         asl_alphabet_test\I_test.jpg           R
9         asl_alphabet_test\J_test.jpg           H
10        asl_alphabet_test\K_test.jpg           F
11        asl_alphabet_test\L_test.jpg           K
12        asl_alphabet_test\M_test.jpg           G
13        asl_alphabet_test\N_test.jpg           N
14        asl_alphabet_test\O_test.jpg           B
15        asl_alphabet_test\P_test.jpg     nothing
16        asl_alphabet_test\Q_test.jpg           W
17        asl_alphabet_test\R_test.jpg       space
18        asl_alphabet_test\S_t

In [15]:
count = 0
for file,prediction in zip(filenames,predictions):
    #print(file,prediction)
    if(prediction+'_test' in file):
        count+=1
        
print("accuracy",count/len(filenames)*100)

accuracy 17.857142857142858


In [ ]:
#https://medium.com/@arindambaidya168/https-medium-com-arindambaidya168-using-keras-imagedatagenerator-b94a87cdefad
#https://medium.com/@vijayabhaskar96/tutorial-image-classification-with-keras-flow-from-directory-and-generators-95f75ebe5720
#

In [7]:
#Define Model VGG16 Model with pretrained weight
def Xception_model_build_weight():
    from keras.applications.xception import Xception
    from keras.layers import Input

    input_tensor = Input(shape=TARGET_DIMS)
    model = Xception(input_tensor = input_tensor, weights='imagenet', include_top=False)
    return model

from keras.models import Model
weight_base_model = Xception_model_build_weight()
x1 = weight_base_model.output
#Add the fully-connected layers 
x1 = Flatten(name='flatten')(x1)
x1 = Dense(4096, activation='relu', name='fc1')(x1)
x1 = Dense(4096, activation='relu', name='fc2')(x1)
x1 = Dense(CLASSES, activation='softmax', name='predictions')(x1)
weight_model = Model(inputs=weight_base_model.input, outputs=x1)

#Compile the model
weight_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

weight_model.summary()
MODEL_PATH = MODEL_DIR+"/Model1-xception-withweight.h5"
weight_model.save(MODEL_PATH)

Instructions for updating:
Colocations handled automatically by placer.
 4448256/83683744 [>.............................] - ETA: 1:21:56

ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [17]:
#Checkpointer to save the best models
MODEL_WEIGHT_PATH = MODEL_DIR+"/Model_Weight1-xception-withweight.h5"
checkpointer = ModelCheckpoint(filepath=MODEL_WEIGHT_PATH, 
                               verbose=1, save_best_only=True)

steps_per_epoch = int( np.ceil(len(train_generator)*2 / BATCH_SIZE) )
validation_steps = int( np.ceil(len(val_generator)*2 / BATCH_SIZE) )

weight_model.fit_generator(train_generator, validation_data=val_generator, 
                    steps_per_epoch =  steps_per_epoch,
                    validation_steps = validation_steps,
                    epochs=10, callbacks=[checkpointer], verbose=1)

Epoch 1/10
39/39 [==============================] - 289s 7s/step - loss: 15.2190 - acc: 0.0361 - val_loss: 15.8159 - val_acc: 0.0187

Epoch 00001: val_loss improved from inf to 15.81588, saving model to ./Model/Model_Weight1-xception-withweight.h5
Epoch 2/10
39/39 [==============================] - 298s 8s/step - loss: 15.5757 - acc: 0.0337 - val_loss: 15.3626 - val_acc: 0.0469

Epoch 00002: val_loss improved from 15.81588 to 15.36256, saving model to ./Model/Model_Weight1-xception-withweight.h5
Epoch 3/10
39/39 [==============================] - 298s 8s/step - loss: 15.5950 - acc: 0.0325 - val_loss: 15.4633 - val_acc: 0.0406

Epoch 00003: val_loss did not improve from 15.36256
Epoch 4/10
39/39 [==============================] - 297s 8s/step - loss: 15.3690 - acc: 0.0465 - val_loss: 15.6648 - val_acc: 0.0281

Epoch 00004: val_loss did not improve from 15.36256
Epoch 5/10
39/39 [==============================] - 300s 8s/step - loss: 15.6273 - acc: 0.0304 - val_loss: 15.5640 - val_acc: 0

In [18]:
#Predict on validation dataset
predictions = weight_model.predict_generator(val_generator, steps=1)        
predictions = np.argmax(predictions, axis=-1) #multiple categories
label_map = (train_generator.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v

predictions = [label_map[k] for k in predictions]

print(predictions, len(predictions))

loss, acc = weight_model.evaluate_generator(val_generator, steps=1, verbose=0)

print(loss,",",acc)

['W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W'] 64
15.614404678344727 , 0.03125


In [14]:
test_image_generator = ImageDataGenerator(
    samplewise_center = True,
    samplewise_std_normalization = True,
)

test_generator = test_image_generator.flow_from_directory(TEST_DIR, target_size=TARGET_SIZE, batch_size=28, shuffle=False, 
    class_mode='categorical')
print(test_generator.classes)

Found 28 images belonging to 1 classes.
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [19]:
#Predict
test_generator.reset()
predictions = weight_model.predict_generator(test_generator, steps=1)
predictions = np.argmax(predictions, axis=1) #multiple categories
label_map = (train_generator.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v

predictions = [label_map[k] for k in predictions]

print(predictions, len(predictions))

['W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W'] 28


In [20]:
filenames=test_generator.filenames
count = 0
for file,prediction in zip(filenames,predictions):
    #print(file,prediction)
    if(prediction+'_test' in file):
        count+=1
        
print("accuracy",count/len(filenames)*100)

accuracy 3.571428571428571
